In [88]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [89]:
data_dir = 'data'

In [90]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path = os.path.join(data_dir,file_name)
    return pd.read_csv(file_path)

In [91]:
flights = get_data("flight_price") 
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [92]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


1. So we have 1 NULL value in Total_stops and 1 in Route. We will later see how to handle these values.
2. Also, some of the columns are not in desired data types so we will have to change them also. 

In [93]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [94]:
flights.Date_of_Journey.iloc[10]

'1/06/2019'

In [95]:
flights.Dep_Time.iloc[10]

'09:45'

In [96]:
flights.Arrival_Time.iloc[10]

'23:00'

In [97]:
flights.Duration.iloc[10]

'13h 15m'

In [98]:
flights.Total_Stops.iloc[19]

'1 stop'

In [99]:
flights.duplicated().sum()

220

In [100]:
(
    flights
    .loc[flights.duplicated(keep=False)]             # to filter out duplicate columns
    .sort_values(['Airline',"Date_of_Journey","Source","Destination"])   # to club all the duplicate columns together
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


1. The Type of Date_of_Journey, Dep_Time and Arrival_Time should be changed to datetime.
2. The type of Duration and Total_Stops is string. It should be numeric.
3. There are 220 Duplicate values which should be removed.

In [101]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [102]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()                         # just to capitalise the first letter of each word 
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

Some of the values in the Airline column are having inaccurate values

In [103]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [104]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [105]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [106]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [107]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [108]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    # .loc[~flights.Duration.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

.loc[lambda ser: ~ser.str.contains("m")]

.loc[~flights.Duration.str.contains("m")]

both the above operations are same and will give same result but when we will perform chaning of all the above steps we should perform the operation on the modified dataset and not on the original one and there we have to go with option 1

In [109]:
(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [110]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


Some of the Values in the Duration column have hour part only. Also, one flight Duration is 5 minutes which is not possible so we should remove that observation

In [111]:
# now we will convert the duration columns values into minutes only
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)  # here we are splitting the duration column which will return a list and then we are expanding which will convert it into a dataframe again with 2 columns one for minute values and other for hour values
    .set_axis(["hour","minutes"],axis=1)  # axis = 1 because we want to assign the header to the column
    .assign(                              # assign method is used to create or update a column
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:(
            df_
            .minutes
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    ) 
    .sum(axis=1)      # our dataframe contains two different columns one for hour and other for minutes we are hust adding them up
    .rename("duration_minutes")
    .to_frame()       # after adding all the columns we get a Series so we are again converting it to dataframe
    .join(flights.Duration)
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [112]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [113]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)  # here we are replacing both stops and stop with empty string and hence we are using regular expression in which ? means if we have stops or stop(string except letter before ?) replace it with empty string
    .pipe(lambda ser :( pd.to_numeric(ser)))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [114]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [115]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand=True)  # here we are splitting the duration column which will return a list and then we are expanding which will convert it into a dataframe again with 2 columns one for minute values and other for hour values
        .set_axis(["hour","minutes"],axis=1)  # axis = 1 because we want to assign the header to the column
        .assign(                              # assign method is used to create or update a column
            hour=lambda df_:(
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minutes=lambda df_:(
                df_
                .minutes
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
        ) 
        .sum(axis=1)      # our dataframe contains two different columns one for hour and other for minutes we are just adding them up
    )

In [116]:
def clean_data(df):
    return(
        df
        .drop(index=6474)
        .drop_duplicates()
        .assign(
            **{
                col: df[col].str.strip()                          # this just removes extra spaces before and after all the observations in the dataframe
                for col in df.select_dtypes(include="O").columns
            }
        )
        .rename(columns=str.lower)
        .assign(
            airline = lambda df_:(   # so airline will be the new column created after the following tranformation
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title() 
            ),
            date_of_journey = lambda df_ : (
                pd.to_datetime(df_.date_of_journey,dayfirst=True)
            ),
            dep_time = lambda df_ : (
                pd.to_datetime(df_.dep_time).dt.time   # .dt.time will extract only time . if we only use .to_datetime it will alloat today's date also
            ),
            arrival_time = lambda df_ : (
                pd.to_datetime(df_.arrival_time).dt.time
            ),
            duration = lambda df_ : (
                convert_to_minutes(df_.duration)
            ),
            total_stops = lambda df_ : (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)  # here we are replacing both stops and stop with empty string and hence we are using regular expression in which ? means if we have stops or stop(string except letter before ?) replace it with empty string
                .pipe(lambda ser :( pd.to_numeric(ser)))
            )
        )
        .drop(columns="route") # we think route will not add extra information to the model and hence drop that
    )

In [117]:
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

# Calculate the number of outliers before applying the Winsorizer
num_outliers_before = count_outliers(flights['Price'])
print(f"Number of outliers before Winsorizing: {num_outliers_before}")

Number of outliers before Winsorizing: 94


In [118]:
from feature_engine.outliers import Winsorizer

winsorizer = Winsorizer(capping_method='iqr', fold=1.5, variables=['Price'])

# Apply the Winsorizer to the DataFrame
flights['Price'] = winsorizer.fit_transform(flights[['Price']])

In [119]:
flights.Price.describe()

count    10683.000000
mean      9021.898156
std       4260.387144
min       1759.000000
25%       5277.000000
50%       8372.000000
75%      12373.000000
max      23017.000000
Name: Price, dtype: float64

In [120]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\91966\AppData\Local\Temp\ipykernel_1528\956307346.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_.dep_time).dt.time   # .dt.time will extract only time . if we only use .to_datetime it will alloat today's date also
C:\Users\91966\AppData\Local\Temp\ipykernel_1528\956307346.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_.arrival_time).dt.time


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No info,12648


In [121]:
flights_cleaned.dtypes

airline                    object
date_of_journey    datetime64[ns]
source                     object
destination                object
dep_time                   object
arrival_time               object
duration                    int64
total_stops               float64
additional_info            object
price                       int64
dtype: object

In [122]:
flights_cleaned = flights_cleaned.dropna()

In [123]:
x = flights_cleaned.drop(columns="price")
y = flights_cleaned['price']

In [124]:
X_, X_test, y_, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6694, 9) (6694,)
(1674, 9) (1674,)
(2093, 9) (2093,)


In [125]:
def export_data(x,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(data_dir,file_name)

    x.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()

In [126]:
export_data(X_train,y_train,"train_")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-27,Delhi,Cochin,20:55:00,12:35:00,940,1.0,In-flight meal not included,12898
1,Jet Airways,2019-06-12,Kolkata,Banglore,18:55:00,16:20:00,1285,1.0,No info,13044
2,Air India,2019-05-18,Delhi,Cochin,09:45:00,09:25:00,1420,2.0,No info,10975
3,Indigo,2019-06-03,Mumbai,Hyderabad,21:20:00,22:50:00,90,0.0,No info,2227
4,Jet Airways,2019-04-01,Mumbai,Hyderabad,02:55:00,04:20:00,85,0.0,No info,5678


In [127]:
export_data(X_test,y_test,"test_")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No info,4462
3,Jet Airways,2019-03-24,Mumbai,Hyderabad,15:50:00,17:20:00,90,0.0,In-flight meal not included,2228
4,Spicejet,2019-04-27,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No info,4991


In [128]:
export_data(X_val,y_val,"val_")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-27,Delhi,Cochin,09:00:00,19:00:00,600,1.0,In-flight meal not included,10675
1,Jet Airways,2019-05-24,Kolkata,Banglore,18:55:00,10:05:00,910,1.0,In-flight meal not included,8586
2,Jet Airways,2019-03-18,Banglore,New Delhi,21:25:00,09:30:00,725,1.0,No info,13555
3,Spicejet,2019-06-27,Chennai,Kolkata,17:45:00,20:05:00,140,0.0,No check-in baggage included,3543
4,Air Asia,2019-05-15,Kolkata,Banglore,07:35:00,19:25:00,710,1.0,No info,5192
